In [ ]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import itertools
import ipywidgets as widgets
from IPython.display import display

# Loading dataset and defining indicators
data = pd.read_csv('../data/fragility_scores.csv')
indicators = ['debt','inflation','reserves','fdi','current_account']

# Defining our robust minmax function
def robust_minmax(x):
    q10, q90 = x.quantile(0.1), x.quantile(0.9)
    return ((x - q10) / (q90 - q10)).clip(0, 1)

In [ ]:
# Grouping data by region and year
region_year = (data.groupby(['year', 'region'])['normalised_fragility_score'].mean().reset_index())

# Creating animated bar chart of average fragility by region
fig = px.bar(
    region_year,
    x = 'region',
    y = 'normalised_fragility_score',
    animation_frame = 'year',
    animation_group = 'region',
    range_y = [0, 1],
    title = 'Average Fragility Score by Region (2010–2021)',
    labels = {'region': 'Region', 'normalised_fragility_score': 'Avg Fragility Score'})

# Adjusting layout for better readability
fig.update_layout(xaxis_tickangle = -39)
fig.show()

In [ ]:
# Reshape data from wide format to long format and convert values to integers
data_long = (data.melt(id_vars = ['country','year'], value_vars = indicators, var_name = 'indicator', value_name = 'value'))
data_long['value'] = pd.to_numeric(data_long['value'], errors='coerce')

# Normalise all values using our robust minmax function
data_long['norm'] = data_long.groupby(['indicator'])['value'].transform(robust_minmax)

# Building a full country-year-indicator grid
grid = pd.DataFrame(itertools.product(
        data_long['country'].unique(),
        data_long['year'].unique(),
        data_long['indicator'].unique()),columns = ['country', 'year', 'indicator'])

# Merging normalised data from data_long into our grid
data_full = (grid.merge(data_long[['country', 'year', 'indicator', 'norm']],on = ['country', 'year', 'indicator'], how = 'left'))

# Fill missing values by forward-fill, backward-fill, then 0 if needed
data_full['norm'] = data_full.groupby(['country', 'indicator'])['norm'].transform(lambda x: x.ffill().bfill().fillna(0))

# Defining all_years for our radar plot
all_years = sorted(data_full['year'].unique())

# Defining parameters for the radar plot
fig = px.line_polar(data_full,
    r = 'norm',
    theta = 'indicator',
    color = 'country',
    line_close = True,
    animation_frame = 'year',
    animation_group = 'country',
    category_orders = {'year': all_years},
    title = 'Fragility Radar Profiles (2010–2021)')

# Improving formatting and readability
fig.update_traces(fill='toself', visible='legendonly')
fig.update_layout(polar=dict(radialaxis=dict(range=[0,1], tickformat=".0%")),legend_title_text='Country')

# Displaying the radar plot
fig

In [ ]:
# Normalising fragility scores using robust minmax
data['fragility_score_robust'] = robust_minmax(data['fragility_score'])

# Defining selectors and output widget
country_selector = widgets.Dropdown(options = sorted(data['country'].unique()), description = 'Country:', layout = widgets.Layout(width = '50%'))
mode_selector = widgets.Dropdown(options = ['Robust Min–Max', 'Standard Min–Max'], description = 'Mode:', layout = widgets.Layout(width = '50%'))
out = widgets.Output()

# Defining the plotting function and data for output modes
def plot_toggle_fragility(country, mode):
    df_ctry = data[data['country'] == country].sort_values('year')
    if mode == 'Robust Min–Max':
        y = df_ctry['fragility_score_robust']
        ylabel = 'Fragility (robust global)'
    else:
        y = df_ctry['normalised_fragility_score']
        ylabel = 'Fragility (standard global)'

    # Update plot based on selected country and mode
    with out:
        out.clear_output(wait = True)
        plt.figure(figsize = (10, 3))
        plt.plot(df_ctry['year'], y, marker = 'o')
        plt.title(f'{country} — {mode}')
        plt.xlabel('Year')
        plt.ylabel(ylabel)
        plt.ylim(0, 1)
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# Setting up interactive plotting
controls = {'country': country_selector, 'mode': mode_selector}
widgets.interactive_output(plot_toggle_fragility, controls)

# Display selectors and plot
display(widgets.VBox([widgets.HBox([country_selector, mode_selector]), out]))

In [ ]:
# Reshape data from wide format to long format
df_long = data.melt(id_vars = ['country', 'year'], value_vars = indicators, var_name = 'indicator', value_name='value')

# Applying robust minmax function to our data
df_long['norm'] = df_long.groupby(['indicator'])['value'].transform(robust_minmax)

# Extracting Zambia's data for 2020
zambia_norm = df_long[(df_long['country'] == 'Zambia') & (df_long['year'] == 2020)]

# Defining parameters for the radar plot
fig = px.line_polar(zambia_norm,
                    r = 'norm',
                    theta = 'indicator',
                    line_close = True,
                    title='Zambia Fragility Profile (2020)')

# Improving formatting and readability
fig.update_traces(fill = 'toself')
fig.update_layout(polar = dict(radialaxis = dict(range = [0, 1], tickformat=".0%")), showlegend = False)

fig.show()

In [ ]:
zambia = data.query("country == 'Zambia'").sort_values('year')

# Defining the plot parameters
plt.figure(figsize = (15, 7))
plt.plot(
    zambia['year'],
    zambia['fragility_score_robust'],
    marker = 'o',
    color = 'crimson',
    label = 'Robust Min–Max Fragility')

# Setting plot title, labels, and improving readability
plt.title('Zambia Fragility Score Over Time (2010–2020)')
plt.xlabel('Year')
plt.ylabel('Normalized Fragility (0–1)')
plt.ylim(0, 1)
plt.xticks(zambia['year'])
plt.legend()
plt.grid(True)

plt.show()